In [13]:
# import dependencies 
import tensorflow as tf
import numpy as np 
import pandas as pd 
from datetime import datetime, date, time
import os 
import glob
import csv

##importing dependencies 
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS 

token = "A1hvR7pmPINW66Wuxbb7dpwR74TukBJIC7IuP3ADhr4xTx0Zzx-JZxzgInUUFbikMsg_bNo2TI4QnbXnUIOkDg=="
org = "ThirdYearProject"
bucket = "86cf3b0e0c4d6c44"

##connect to influxdb
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org,timeout=300000)

##DATA INTO INFLUXDB
##creating object of write API 
##synchronous specifies that there will be multiple rows of data being stored
##at a time
write_api = client.write_api(write_options=SYNCHRONOUS,batch_size=10000,flush_inveral=5000)

file_list=[]

data = pd.read_csv(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\allCSV\channel_1001_segment_28.csv",index_col=None,names=col_names)

data[['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']]=data['HEADER'].apply(lambda x: pd.Series(str(x).split("\t")))
data.drop('HEADER', axis=1,inplace=True)
convert_dict={'Q-Factor':float,
             'Transmit Power (dBm)':float,
             'Chromatic Dispersion(ps/nm)':float
             }
data=data.astype(convert_dict)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36160 entries, 0 to 36159
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   DateTime                      36160 non-null  object 
 1   Q-Factor                      36160 non-null  float64
 2   Transmit Power (dBm)          36160 non-null  float64
 3   Chromatic Dispersion(ps/nm)   36160 non-null  float64
 4   Polarization Mode Dispersion  36160 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1.4+ MB


In [15]:
col_names=['HEADER']
csv_directory=r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\allCSV"
for filename in os.listdir(csv_directory):
    file_list.append(filename)
    data = pd.read_csv(csv_directory+'/'+filename,index_col=None,names=col_names)

    #giving column titles to data
    #data.columns = ['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']
    data[['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']]=data['HEADER'].apply(lambda x: pd.Series(str(x).split("\t")))
    data.drop('HEADER', axis=1,inplace=True)
    data['Polarization Mode Dispersion'] = data['Polarization Mode Dispersion'].replace('?',0)
    convert_dict={'Q-Factor':float,
                 'Transmit Power (dBm)':float,
                 'Chromatic Dispersion(ps/nm)':float,
                  'Polarization Mode Dispersion':float
                 }
    data=data.astype(convert_dict)
    #clearning up data -> replacing ? with 0 (maybe should replace it with the average)
    #data['Polarization Mode Dispersion'] = pd.to_numeric(data['Polarization Mode Dispersion'], downcast="float",errors='coerce')
    #data.fillna(0)
    data['DateTime'] = pd.to_datetime(data['DateTime'], format="%Y.%m.%d.%H.%M.%S")
    data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    data.set_index(data['DateTime'], drop = True, inplace = True)

    #data.drop('DateTime', axis=1, inplace=True)
    data.drop('DateTime', axis=1, inplace=True)
    
    print(data['Polarization Mode Dispersion'].dtype)
    
    ##writing data into influxdb
    response = write_api.write(bucket, record=data, data_frame_measurement_name='Network Prediction')

float64
float64
float64


KeyboardInterrupt: 

In [16]:
print(file_list)

['channel_1000_segment_28.csv', 'channel_1001_segment_28.csv', 'channel_1002_segment_28.csv', 'channel_1003_segment_28.csv', 'channel_1004_segment_28.csv', 'channel_1005_segment_28.csv', 'channel_1006_segment_28.csv', 'channel_1007_segment_28.csv', 'channel_1008_segment_28.csv', 'channel_1009_segment_28.csv', 'channel_100_segment_3.csv', 'channel_1010_segment_28.csv', 'channel_1011_segment_28.csv', 'channel_1012_segment_28.csv', 'channel_1013_segment_28.csv', 'channel_1014_segment_28.csv', 'channel_1015_segment_28.csv', 'channel_1016_segment_28.csv', 'channel_1017_segment_28.csv', 'channel_1018_segment_28.csv', 'channel_1019_segment_28.csv', 'channel_101_segment_4.csv', 'channel_1020_segment_28.csv', 'channel_1021_segment_28.csv', 'channel_1022_segment_28.csv', 'channel_1023_segment_28.csv', 'channel_1024_segment_28.csv', 'channel_1025_segment_28.csv', 'channel_1026_segment_28.csv', 'channel_1027_segment_28.csv', 'channel_1028_segment_28.csv', 'channel_1029_segment_28.csv', 'channel_10

In [17]:
len(file_list)

939

In [23]:
import csv

In [24]:
rows = [[item] for item in file_list]
print(rows)
file = open(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv", 'a+',newline ='')
with file:
    write = csv.writer(file)
    write.writerows(rows)

[['channel_1000_segment_28.csv'], ['channel_1001_segment_28.csv'], ['channel_1002_segment_28.csv'], ['channel_1003_segment_28.csv'], ['channel_1004_segment_28.csv'], ['channel_1005_segment_28.csv'], ['channel_1006_segment_28.csv'], ['channel_1007_segment_28.csv'], ['channel_1008_segment_28.csv'], ['channel_1009_segment_28.csv'], ['channel_100_segment_3.csv'], ['channel_1010_segment_28.csv'], ['channel_1011_segment_28.csv'], ['channel_1012_segment_28.csv'], ['channel_1013_segment_28.csv'], ['channel_1014_segment_28.csv'], ['channel_1015_segment_28.csv'], ['channel_1016_segment_28.csv'], ['channel_1017_segment_28.csv'], ['channel_1018_segment_28.csv'], ['channel_1019_segment_28.csv'], ['channel_101_segment_4.csv'], ['channel_1020_segment_28.csv'], ['channel_1021_segment_28.csv'], ['channel_1022_segment_28.csv'], ['channel_1023_segment_28.csv'], ['channel_1024_segment_28.csv'], ['channel_1025_segment_28.csv'], ['channel_1026_segment_28.csv'], ['channel_1027_segment_28.csv'], ['channel_102